## Setup
Click *Run all*, and then skip to the final cell (bottom of the notebook). 
If there are errors before you get to the final cell, let me know.

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from matplotlib import rcParams 
import ipywidgets as widgets 
import io 
from pathlib import Path 
from IPython.display import display 

from tkinter import Tk
from tkinter import filedialog

In [2]:
# create dummy data 

# dummy_data = {
#     'Total number of abortion providers' : [5, 24, 4, 5, 4, 1, 11, 1, 2, 38, 49, 3, 1],
#     'Number of rural abortion providers' : [0, 1, 0, 0, 1, 0, 9, 0, 1, 4, 6, 0, 0]
# }
# dummy_data = pd.DataFrame(
#     dummy_data, 
#     index=[
#         'Alberta', 'British Columbia', 'Manitoba', 
#         'New Brunswick', 'Newfoundland and Labrador', 
#         'Northwest Territories', 'Nova Scotia', 
#         'Prince Edward Island', 'Nunavut', 'Ontario', 
#         'Quebec', 'Saskatchewan', 'Yukon'
#     ]
# )

# dummy_data.to_csv(Path.cwd().parent / 'data/total_and_rural_abortion_providers_by_province.csv')

## Upload the data file.

**Requirements**
- It must be in **.csv** format; otherwise, open it in Excel/Google sheets, then "Save as.." .csv
- The first column should be row titles. If you do not use row titles, then replace these with numbers $0, 1, \dots, n$ where $n$ is the number of rows.
- The first row should be column titles. The first column does not require a title, but you can add one if you wish.
- Besides the first column, there should be 2 other columns to the right, i.e. the data should be of dimension $(n \times 3)$ (rows x columns)

### Instructions
- Go to the 'data' folder (click the folder icon in the left sidebar) and upload your folder and give it a **unique** name
- In the cell below, replace `None` with the **full name** of the CSV file you just added (i.e. include the extension)

In [3]:
upload = None

## Process the data

In [4]:
# root = Tk()
# root.withdraw()
# root.call('wm', 'attributes', '.', '-topmost', True)
# upload = filedialog.askopenfilename(defaultextension=".csv", title="Choose data (.CSV) file.")

homedir = Path.cwd().parent 
try:
    df = pd.read_csv(
        io.BytesIO(upload),
        header=0, index_col=0,
    )
    save_path = homedir / "output/{upload[:-4]}.png"
except Exception:
    print("No upload was found. Using test data..")
    save_path = homedir / "output/test.png"
    
    df = pd.read_csv(
        homedir / 'data/total_and_rural_abortion_providers_by_province.csv',
        index_col=0, header=0
    )

df.head()

No upload was found. Using test data..


,Total number of abortion providers,Number of rural abortion providers
Alberta,5,0
British Columbia,24,1
Manitoba,4,0
New Brunswick,5,0
Newfoundland and Labrador,4,1


## Widget definitions.

In [5]:
plt.style.use('default')
class MirroredBars:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.create_figure()

    def create_figure(self, **kwargs):
        self.fig, self.ax = plt.subplots(
            1, 1, figsize=(9, 6),
            constrained_layout=True,
            **kwargs
        )
        
    def sort_data(self, by=0, ascending=True):
        
        if isinstance(by, int):
            by = self.data.columns[by]
        
        self.data.sort_values(
            by=by, ascending=ascending,
            inplace=True
        )

    def _add_bars(self, yvals: np.ndarray, width: pd.Series, max_height: float, **kw):
        
        self.ax.barh(
            yvals, 
            width,
            max([0.5, max_height]),
            **kw
        )

    def _add_data_labels(
        self, left: int, right: int, 
        left_color: str, right_color: str, dx: float
    ):
        left_font = dict(
            fontsize=12, fontweight='bold',
            color=left_color, ha='right', va='center',
            # transform=self.ax.transData
        )
        right_font = left_font.copy()
        right_font['color'] = right_color

        for i in range(self.data.shape[0]):
            x1, x2 = self.data.iloc[i,[left, right]]
            
            self.ax.text(
                -x1 - dx*0.4, i, x1, 
                fontdict=left_font,
            )

            self.ax.text(
                x2 + dx, i, x2, 
                fontdict=right_font
            )

    def equalize_xlims(self, xlims: tuple, min_xlim_frac=None):
        if min_xlim_frac:

            a, b = np.abs(xlims)
            
            r = a / (a+b)
            if r < min_xlim_frac:
                da = (a*(min_xlim_frac-1) + b*min_xlim_frac)/(1 - min_xlim_frac)
                self.ax.set_xlim([xlims[0] - da, xlims[1]])
                return 
            
            if (1-r) < min_xlim_frac:
                db = (b*(min_xlim_frac-1) + a*min_xlim_frac)/(1 - min_xlim_frac)
                self.ax.set_xlim([xlims[0], xlims[1]+db])
                return 
        
        if xlims[1] > abs(xlims[0]):
            self.ax.set_xlim(
                [-xlims[1], xlims[1]]
            )
        else:
            self.ax.set_xlim(
                [xlims[0], abs(xlims[0])]
            )

    def _hide_spines(self, spines: list):
        for spine in spines:
            self.ax.spines[spine].set_visible(False)

    def _num_xbins(self, n: int):
        self.ax.locator_params(axis='x', nbins=n)

    def plot(
        self, 
        left=0, right=1, 
        left_color='red', right_color='blue',
        left_edgecolor='red', right_edgecolor='blue',
        add_bar_labels=True, text_frac=0.03, 
        equal_xlims=True, min_xlim_frac=None,
    ) -> plt.Axes:
        yvals = np.arange(self.data.shape[0], dtype=int)

        max_height = (max(yvals) - min(yvals)) / (yvals.shape[0] + 1)

        self._add_bars(
            yvals, self.data.iloc[:, left]*-1, 
            max_height, color=left_color,
            edgecolor=left_edgecolor,
            label=self.data.columns[left]
        )
        
        self._add_bars(
            yvals, self.data.iloc[:, right], 
            max_height, color=right_color,
            edgecolor=right_edgecolor,
            label=self.data.columns[right]
        )

        self.ax.set_yticks(yvals)
        self.ax.set_yticklabels(self.data.index)

        xlims = self.ax.get_xlim()
        if equal_xlims or min_xlim_frac is not None:
            self.equalize_xlims(xlims, min_xlim_frac=min_xlim_frac)
        
        if add_bar_labels:
            dx = (xlims[1] - xlims[0]) * text_frac
            self._add_data_labels(
                left, right, left_color, right_color, dx
            )
            
        xticks = self.ax.get_xticks()
        self.ax.set_xticks(xticks)
        xticklabs = [abs(x) if x < 0 else x for x in xticks]
        self.ax.set_xticklabels(xticklabs)

        return self.ax 

In [6]:
style = {'description_width' : 'initial'}
colorpicker_kw = dict(value='blue', concise=False, disabled=False, style=style)
checkbox_kw = dict(value=False, disabled=False, indent=False, style=style)

In [7]:
fontsizes = [
    widgets.BoundedFloatText(
        value=12, min=2, max=100, step=0.1, description=lab, disabled=False
    ) for lab in [
        'Other text', 'X-tick labels', 'Y-tick labels', 
        'Axes label', 'Axes title', 'Legend'
    ]
]

color_pickers = {
    f"{side} color" : widgets.ColorPicker(
        description=f"Pick the {side} bar color", 
        **colorpicker_kw
    ) for side in ["LEFT", "RIGHT"]
}
color_pickers.update({
    f"{side} edgecolor" : widgets.ColorPicker(
        description=f"Pick the color of the {side} bar edges",
        **colorpicker_kw
    ) for side in ["LEFT", "RIGHT"]
})
color_pickers["Axes facecolor"] = widgets.ColorPicker(
    description="Pick the color of the axes",
    **colorpicker_kw
)

customs = {}
customs['Data label distance'] = widgets.BoundedFloatText(
    value=0.03, min=0, max=1, step=0.01, 
    description='Distance between bar and data label', 
    disabled=False, style=style
)

customs['Number of X-ticks'] = widgets.BoundedIntText(
    value=5, min=3, max=10, step=1,
    description='Number of X-ticks (rough)',
    disabled=False, style=style
)

customs['Left/right spacing'] = widgets.Checkbox(
    description="Modify space occupied by left/right bars",
    **checkbox_kw
)

customs['Minimum left/right fraction'] = widgets.BoundedFloatText(
    value=0.3, min=0, max=0.49, step=0.01, style=style,
    description = "Minimum space occupied by left/right bars"
)

customs['Add data labels'] = widgets.Checkbox(
    description="Add data labels",
    **checkbox_kw
)

customs['Add legend'] = widgets.Checkbox(
    description='Show legend', **checkbox_kw
)
customs['Legend location'] = widgets.Dropdown(
    options=[
        'best', 'upper right', 'center right', 'upper left',
        'lower left', 'lower center', 'lower right'
    ],
    value='upper right',
    description="Legend location",
    disabled=False, style=style,
)

customs['Hide spines'] = widgets.SelectMultiple(
    options=['top', 'bottom', 'right'],
    value=['top', 'right'],
    description='Spines to hide',
    disabled=False, style=style
)

customs['Spine linewidth'] = widgets.BoundedFloatText(
    value=1.0, min=0.1, max=5.0, step=0.1,
    description="Width of axes spines", style=style
)

customs.update({f"{side} index" : widgets.BoundedIntText(
        value=0, min=0, max=None, step=1, 
        description=f"{side} index",
        style=style
    ) for side in ["LEFT", "RIGHT"]
})

customs['Save figure'] = widgets.Checkbox(
    description="Save the figure",
    **checkbox_kw
)

In [8]:
tab_names = [
    'Data', 'Font size', 'Colors', 'Axes elements', 'Create the plot'
]

TAB = widgets.Tab()

lay = widgets.Layout(grid_template_columns="repeat(3, 400px)")
data_grid = widgets.GridBox(
    [
        widgets.Label("Data options"), customs['LEFT index'], customs['RIGHT index'],
    ], layout=lay
)

# fontsizes.insert(0, widgets.Label("Text sizes"))
fs_grid = widgets.GridBox(
    fontsizes, layout=lay
)

clrs = list(color_pickers.values())
clrs.insert(0, widgets.Label('Colors'))
clrs_grid = widgets.GridBox(clrs, layout=lay)

axes_grid = [
    'Add data labels', 'Data label distance', 
    'Number of X-ticks', 'Left/right spacing',  'Minimum left/right fraction', 
    'Add legend', 'Spine linewidth', 'Hide spines',
    'Legend location', 
    
]
axes_grid = [customs[k] for k in axes_grid]
axes_grid.insert(0, widgets.Label('Axes options'))
axes_grid = widgets.GridBox(
    axes_grid, layout=lay
)

wOutput = widgets.Output()
def parse_widget_inputs(b):
    with wOutput:
        mirror = MirroredBars(df)
        sides = ['LEFT', 'RIGHT']
        
        fs_keys = [
            'Other text', 'X-tick labels', 'Y-tick labels', 
            'Axes label', 'Axes title', 'Legend'
        ]

        sel = {k : v.value for k, v in zip(fs_keys, fontsizes)}
        sel.update({k : v.value for k, v in color_pickers.items()})
        sel.update({k : v.value for k, v in customs.items()})
        print(sel['Hide spines'])

        left_fc, right_fc = [sel[f'{s} color'] for s in sides]
        left_ec, right_ec = [sel[f'{s} edgecolor'] for s in sides]
        
        rcParams['axes.facecolor'] = sel['Axes facecolor'] 
        rcParams['figure.dpi'] = 300 
        print("Figure DPI is set to 300 by default.\nYou may want to scale font sizes down by 3.")

        rcParams['font.size'] = sel['Other text'] 
        rcParams['axes.labelsize'] = sel['Axes label'] 
        rcParams['xtick.labelsize'] = sel['X-tick labels'] 
        rcParams['ytick.labelsize'] = sel['Y-tick labels'] 
        rcParams['axes.titlesize'] = sel['Axes title'] 
        rcParams['legend.fontsize'] = sel['Legend'] 

        rcParams['axes.linewidth'] = sel['Spine linewidth'] 
        if sel['Hide spines']:
            for spine in sel['Hide spines']:
                rcParams[f'axes.spines.{spine}'] = False 
        
        ax = mirror.plot(
            left=sel['LEFT index'], right=sel['RIGHT index'],
            left_color=left_fc, right_color=right_fc,
            left_edgecolor=left_ec, right_edgecolor=right_ec,
            add_bar_labels=sel['Add data labels'], 
            text_frac=sel['Data label distance'],
            equal_xlims=sel['Left/right spacing'],
            min_xlim_frac=sel['Minimum left/right fraction'],
        )

        if sel['Add legend']:
            ax.legend(sel['Legend location'])
        
        if sel['Save figure']:
            plt.savefig(save_path, dpi=300)
            print(f"The figure was successfully saved at {save_path}")

        plt.show()


button = widgets.Button(
    description="RUN", disabled=False,
    # button_style='success',
    tooltip='Create the plot',
    icon='check'
)
button.on_click(parse_widget_inputs)

output_grid = widgets.GridBox(
    [button, customs['Save figure']],
    layout=lay
)

TAB.children = [
    data_grid, fs_grid, 
    clrs_grid, axes_grid, 
    output_grid
]

for i, t in enumerate(tab_names):
    TAB.set_title(i, t)


## Select appearance properties.

In [9]:
display(wOutput)
display(TAB)

Output()

### Instructions
- Go through the tabs above and fill in the appropriate values.
- The figure has dpi 300, which means you may want to scale down the font sizes by 3. 
- For 'Colors,' note that the default colors for the axes, bars, and bar edges are **all blue.** If you do not change the axes color, *the whole plot will look blue.*
- For 'Axes options/elements,' you generally want to *Add data labels* and *Modify space occupied by left/right bars*
  - *Add data labels*: adds numeric values on top of each bar
  - *Modify space occupied by...*: expands the X-axis so that each side occupies some minimum fraction of the X-axis. The fraction is determined by the *Minimum space occupied by...* parameter. I recommend 0.2 or 0.3. Note that **this has no effect** if the data already meets or exceeds the fraction you've set.
- *Create the plot*
  - When you press *RUN*, any output (e.g. error messages) will appear in the cell above. If you have any problems, feel free to ping me and send a screenshot of what errors you encountered, as well as the settings you used.
  - *Save the figure.* You generally want to test a couple settings before saving, but, once you want to save, check this box and select a location on your computer to save the figure at. It should save a *.png* at the corresponding location.